In [ ]:
try:
    import transformers
except ImportError:
    !pip install transformers
    import transformers
try:
    import emoji
except ImportError:
    !pip install emoji
    import emoji

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.0 MB/s 
     |████████████████████████████████| 101 kB 10.4 MB/s 
     |████████████████████████████████| 6.6 MB 50.3 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 197 kB 6.1 MB/s 
  Created wheel for emoji: filename=emoji-2.0.0-py3-none-any.whl size=193022 sha256=a5f76174150565f9dd2eeafd2c945bc1dc16a554681a13eb912382dc51b01976
  Stored in directory: /root/.cache/pip/wheels/ec/29/4d/3cfe7452ac7d8d83b1930f8a6205c3c9649b24e80f9029fc38
Successfully built emoji


In [ ]:
local_run = False
local_path = "C:\\Greenwich\\MSc Project\\project_code\\"
store_local = False

In [ ]:
import pandas as pd
import numpy as np

import random
import string
import re
import ast

from scipy.special import softmax
import csv
import urllib.request

from sklearn.model_selection import train_test_split
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TFAutoModelForSequenceClassification, AutoConfig
from transformers import TFDistilBertForSequenceClassification, TrainingArguments, Trainer
from transformers import DataCollatorWithPadding

from sklearn.metrics import confusion_matrix,f1_score

In [ ]:
full_train_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_train_data.csv?raw=true")
test_data = pd.read_csv("https://github.com/Voldegin/hate_speech_detection/blob/develop/data/mixed/mixed_test_data.csv?raw=true")

In [ ]:
train_data, val_data = train_test_split(full_train_data,test_size=5000,random_state=21)

In [ ]:
len(train_data), len(val_data), len(test_data)

(84979, 5000, 10000)

**Splitting into labels and features**

In [ ]:
# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
def split_label_and_feature(data):
    data['cleaned'] = data['tweet_text'].apply(preprocess)
    return data['cleaned'], data['is_cyberbullying']

In [ ]:
X_train, y_train = split_label_and_feature(train_data)
X_val, y_val = split_label_and_feature(val_data)
X_test, y_test = split_label_and_feature(test_data)

In [ ]:
X_train.head()

51990    "\n Yes, it is. But a demo is not notable enou...
5095     Has it occurred to you that I wouldn't tell so...
64041    Notice of Edit warring noticeboard discussion\...
53391    ":It doesn't even seem that an amendment is su...
10       "\n\nUNBLOCK ME OR I'LL GET MY LAWYERS ON TO Y...
Name: cleaned, dtype: object

In [ ]:
# task='sentiment'
# MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

# tokenizer = AutoTokenizer.from_pretrained(MODEL)

# # download label mapping
# labels=[]
# mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
# with urllib.request.urlopen(mapping_link) as f:
#     html = f.read().decode('utf-8').split("\n")
#     csvreader = csv.reader(html, delimiter='\t')
# labels = [row[1] for row in csvreader if len(row) > 1]

# # PT
# model = AutoModelForSequenceClassification.from_pretrained(MODEL)
# # model.save_pretrained(MODEL)

In [ ]:
# def predict_cardiff_model(text):
#     # print(text)
#     text = preprocess(text)
#     encoded_input = tokenizer(text, return_tensors='pt',max_length = 512,truncation = True)
#     output = model(**encoded_input)
#     scores = output[0][0].detach().numpy()
#     scores = softmax(scores)

#     ranking = np.argsort(scores)
#     ranking = ranking[::-1]
#     # print(ranking)
#     # print(scores)
#     # print(scores.shape[0])
#     # print(scores.argmax())
#     pred = scores.argmax()
#     if pred == 0:
#         return 1
#     else:
#         return 0
#     # for i in range(scores.shape[0]):
#     #     l = labels[ranking[i]]
#     #     s = scores[ranking[i]]
#     #     print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def predict_cardiff_model(text):
    text = preprocess(text)
    encoded_input = tokenizer(text, return_tensors='pt',max_length = 512,truncation = True)
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    # # TF
    # model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
    # model.save_pretrained(MODEL)
    # text = "Covid cases are increasing fast!"
    # encoded_input = tokenizer(text, return_tensors='tf')
    # output = model(encoded_input)
    # scores = output[0][0].numpy()
    # scores = softmax(scores)
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    # pred = scores.argmax()
    # if pred == 0:
    #     return 1
    # else:
    #     return 0
    for i in range(scores.shape[0]):
        l = config.id2label[ranking[i]]
        s = scores[ranking[i]]
        print(f"{i+1}) {l} {np.round(float(s), 4)}")

In [ ]:
predict_cardiff_model("Muslims are terrorists")

1) Negative 0.7504
2) Neutral 0.2245
3) Positive 0.0251


In [ ]:
predict_cardiff_model("Good Day")

1) Positive 0.9149
2) Neutral 0.0699
3) Negative 0.0152


In [ ]:
val_preds = X_val.apply(predict_cardiff_model)
val_preds

3396     0
28135    1
41399    1
62205    0
1432     0
        ..
6627     0
55882    0
65333    0
26645    0
66400    0
Name: cleaned, Length: 5000, dtype: int64

In [ ]:
confusion_matrix(y_val,val_preds)

array([[1536,  957],
       [ 458, 2049]])

In [ ]:
f1_score(y_val,val_preds)

0.7433339379648104

In [ ]:
test_preds = X_test.apply(predict_cardiff_model)
test_preds

0       0
1       1
2       1
3       0
4       0
       ..
6989    1
6990    1
6991    1
6992    1
6993    1
Name: cleaned, Length: 6994, dtype: int64

In [ ]:
confusion_matrix(y_test,test_preds)

array([[2354, 1145],
       [ 365, 3130]])

In [ ]:
f1_score(y_test,test_preds)

0.8056628056628056